## Installing TensorFlow

In [3]:
!pip install tensorflow

  Using cached tensorflow-2.13.0-cp39-cp39-win_amd64.whl (1.9 kB)


## Importing Libraries

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

## Read The Text File

In [8]:
with open('stories.txt', 'r', encoding='utf-8') as file:
    text = file.read()

## Tokenize the text to create a sequence

In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

## creating input-outputs by splitting the text into sequences of tokens and forming n-grams from the sequences

In [12]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

## Padding


In [13]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

## Input And Output

In [14]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

## Coverting to encode vectors

In [15]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

## Neural Network Architecture

In [16]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


## Train the model

In [ ]:
#it will take some time because the learning rate of adam is 0.001
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 111s 36ms/step - loss: 6.2331 - accuracy: 0.0770
Epoch 2/100
3010/3010 [==============================] - 109s 36ms/step - loss: 5.4993 - accuracy: 0.1249
Epoch 3/100
3010/3010 [==============================] - 106s 35ms/step - loss: 5.1112 - accuracy: 0.1484
Epoch 4/100
3010/3010 [==============================] - 113s 38ms/step - loss: 4.7847 - accuracy: 0.1648
Epoch 5/100
3010/3010 [==============================] - 108s 36ms/step - loss: 4.4853 - accuracy: 0.1831
Epoch 6/100
3010/3010 [==============================] - 106s 35ms/step - loss: 4.2023 - accuracy: 0.2034
Epoch 7/100
3010/3010 [==============================] - 105s 35ms/step - loss: 3.9316 - accuracy: 0.2285
Epoch 8/100
3010/3010 [==============================] - 102s 34ms/step - loss: 3.6742 - accuracy: 0.2575
Epoch 9/100
3010/3010 [==============================] - 104s 35ms/step - loss: 3.4346 - accuracy: 0.2913
Epoch 10/100
3010/3010 [======================

## Lets Start!

In [ ]:
seed_text = "I will leave if they"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)